Notebook para mezclar los tres conjuntos de datos: las bandas Ku y Ka, y ldpr (en version EASEv2 los tres datasets)

In [1]:
import os
from datetime import datetime
import zipfile
import xarray as xr  
import rasterio
import numpy as np
import io

In [2]:
def scale_factor_correction(input_file):
    corrected_ds = xr.Dataset()
    input_ds = xr.open_dataset(input_file, engine='netcdf4')

    for var in input_ds.data_vars:
        data_array = input_ds[var]
        if 'SCALE FACTOR' in data_array.attrs:
            scale_factor = data_array.attrs['SCALE FACTOR']
            corrected_ds[var] = data_array * scale_factor
        else:
            corrected_ds[var] = data_array

    return corrected_ds

def get_day_of_year(file_path):
    if 'LPDR_v3_2017_EASEv2_h5_landmask' in file_path:
        year_doyy_str = file_path.split('/')[-1].split('_')[2][:7]
        year_doyy = year_doyy_str[-3:]
    else:
        date_str = file_path.split('/')[-1].split('_')[1]
        datetime_obj = datetime.strptime(date_str, '%Y%m%d')
        year_doyy = datetime_obj.strftime('%Y%j')[4:]
    
    return year_doyy

def get_orientation(file_path): 
    if 'LPDR_v3_2017_EASEv2_h5_landmask' in file_path:
        orientation_str = file_path.split('/')[-1].split('_')[2][:8]
        orientation = orientation_str[-1:]
    else:
        orientation_str = file_path.split('/')[-1].split('_')[3]
        orientation = orientation_str[-1:]

    return orientation



def process_ldpr_dataset(lpdr_file):
    lpdr_ds = xr.open_dataset(lpdr_file, engine='netcdf4')
    
    df = lpdr_ds.to_dataframe()
    df = df.reset_index()
    
    df_pivot = df.pivot(index=['lat', 'lon', 'landmask'], columns='band', values='band_data')  
    
    df_pivot.columns = ['fwns', 'fw', 'air temperature', 'PWV', 'VOD', 'vsm', 'VPD', '7', '8']
    df_pivot = df_pivot.drop(columns=['7', '8'])
    df_pivot.dropna(inplace=True)
    
    ds = df_pivot.reset_index().set_index(['lat', 'lon']).to_xarray()
    
    return ds



def process_satellite_data(lpdr_file, ka_file, ku_file):
    lpdr_ds = process_ldpr_dataset(lpdr_file)
    
    ka_ds = scale_factor_correction(ka_file)
    ku_ds = scale_factor_correction(ku_file)
    
    ka_ds = ka_ds.rename({'Brightness Temperature (H)': 'Brightness Temperature (H)_ka',
                          'Brightness Temperature (V)': 'Brightness Temperature (V)_ka'})
    
    ku_ds = ku_ds.rename({'Brightness Temperature (H)': 'Brightness Temperature (H)_ku',
                          'Brightness Temperature (V)': 'Brightness Temperature (V)_ku'})
    
    
    land_mask_transformed = lpdr_ds.copy()
    land_mask_transformed = land_mask_transformed.assign_coords({'lat': ku_ds['lat'].values, 'lon': ku_ds['lon'].values})
    lpdr_ds_final = land_mask_transformed.rename({"lon": "lon", "lat": "lat"})
    
    lpdr_ds_final = lpdr_ds_final.assign(lat_var=lpdr_ds_final['lat'], lon_var=lpdr_ds_final['lon'])
    merged_ds = xr.merge([lpdr_ds_final, ku_ds, ka_ds], compat='override')
    
    return merged_ds

def process_final_dataset_to_dataframe(final_ds):
    df = final_ds.to_dataframe()
    
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('(', '_')
    df.columns = df.columns.str.replace(')', '')
    
    df.replace(-999.0, np.nan, inplace=True)
    df.dropna(inplace=True)
    
    df = df[df['landmask'] == 1]
    df.drop(columns='landmask', inplace=True)
    
    df.reindex()
    
    return df

lpdr_dir = "../data/LPDR_v3_2017_EASEv2_h5_landmask/"
ku_dir = "../data/ku_18ghz/"
ka_dir = "../data/ka_36ghz/"

def find_corresponding_lpdr_files(lpdr_dir, ku_dir, ka_dir):
    lpdr_files = []

    for lpdr_file in os.listdir(lpdr_dir):
        if lpdr_file.endswith(".h5"):
            lpdr_file_path = os.path.join(lpdr_dir, lpdr_file)
            lpdr_doyy = get_day_of_year(lpdr_file_path)
            lpdr_orientation = get_orientation(lpdr_file_path)

            for ku_file in os.listdir(ku_dir):
                if ku_file.endswith(".h5"):
                    ku_file_path = os.path.join(ku_dir, ku_file)
                    ku_doyy = get_day_of_year(ku_file_path)
                    ku_orientation = get_orientation(ku_file_path)

                    if lpdr_doyy == ku_doyy and ku_orientation == lpdr_orientation:
                        for ka_file in os.listdir(ka_dir):
                            if ka_file.endswith(".h5"):
                                ka_file_path = os.path.join(ka_dir, ka_file)
                                ka_doyy = get_day_of_year(ka_file_path)
                                ka_orientation = get_orientation(ka_file_path)

                                if (lpdr_doyy == ka_doyy == ku_doyy) and \
                                (lpdr_orientation == ka_orientation == ku_orientation):
                                    lpdr_files.append((lpdr_file_path, ku_file_path, ka_file_path, lpdr_doyy, lpdr_orientation))

    return lpdr_files

corresponding_files = find_corresponding_lpdr_files(lpdr_dir, ku_dir, ka_dir)

output_dir = "../data/"
zip_file_path = os.path.join(output_dir, "processed_data.zip")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for lpdr_file, ku_file, ka_file, lpdr_doyy, lpdr_orientation in corresponding_files:
        print('lpdr_file', lpdr_file)
        print('ku_file', ku_file)
        print('ka_file', ka_file)
        final_dataset = process_satellite_data(lpdr_file, ka_file, ku_file)
        processed_dataframe = process_final_dataset_to_dataframe(final_dataset)
        
        csv_buffer = io.StringIO()
        processed_dataframe.to_csv(csv_buffer, index=False)
        csv_filename = f'processed_data_{lpdr_doyy}_{lpdr_orientation}.csv'
        zipf.writestr(csv_filename, csv_buffer.getvalue())

print(f'Processed {len(corresponding_files)} files and saved to {zip_file_path}')

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017352D.h5
ku_file ../data/ku_18ghz/GW1AM2_20171218_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171218_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017142A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170522_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170522_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017346D.h5
ku_file ../data/ku_18ghz/GW1AM2_20171212_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171212_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017156A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170605_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170605_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017355A.h5
ku_file ../data/ku_18ghz/GW1AM2_20171221_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171221_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017151D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170531_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170531_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017341A.h5
ku_file ../data/ku_18ghz/GW1AM2_20171207_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171207_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017179D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170628_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170628_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017020A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170120_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170120_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017218D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170806_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170806_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017008A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170108_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170108_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017236A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170824_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170824_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017232A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170820_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170820_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017024A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170124_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170124_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017234D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170822_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170822_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017030A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170130_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170130_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017345A.h5
ku_file ../data/ku_18ghz/GW1AM2_20171211_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171211_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017155D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170604_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170604_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017351A.h5
ku_file ../data/ku_18ghz/GW1AM2_20171217_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171217_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017141D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170521_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170521_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

...

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017116D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170426_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170426_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017306A.h5
ku_file ../data/ku_18ghz/GW1AM2_20171102_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171102_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017138A.h5
ku_file ../data/ku_18ghz/GW1AM2_20170518_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170518_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017328D.h5
ku_file ../data/ku_18ghz/GW1AM2_20171124_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171124_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017164D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170613_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170613_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017170D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170619_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170619_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017360A.h5
ku_file ../data/ku_18ghz/GW1AM2_20171226_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171226_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017158D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170607_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170607_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017198D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170717_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170717_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017173D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170622_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170622_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017363A.h5
ku_file ../data/ku_18ghz/GW1AM2_20171229_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171229_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017167D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170616_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170616_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017016D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170116_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170116_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017189D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170708_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170708_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017176D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170625_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170625_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017162D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170611_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170611_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20

lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017001D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170101_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170101_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017149D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170529_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170529_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017359A.h5
ku_file ../data/ku_18ghz/GW1AM2_20171225_01D_EQMA_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20171225_01D_EQMA_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_2017161D.h5
ku_file ../data/ku_18ghz/GW1AM2_20170610_01D_EQMD_L3SGT18LA2220220_corrected.h5
ka_file ../data/ka_36ghz/GW1AM2_20170610_01D_EQMD_L3SGT36LA2220220_corrected.h5
lpdr_file ../data/LPDR_v3_2017_EASEv2_h5_landmask/AMSRU_Mland_20